## Install geemap

In [9]:
#!pip install geemap

In [10]:
import ee
#import geemap

In [ ]:
#Map = geemap.Map()
#Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Extract Precipitation

### Constants:

In [11]:
# Ref: https:#developers.google.com/earth-engine/tutorials/community/ph-ug-temp

'Municipalities:'
# Import country boundaries feature collection.
dataset = ee.FeatureCollection('users/dsrestrepo/mpio')
Codes = dataset.aggregate_array('MPIOS')

'Dates:'
# Define a date range of interest; here, a start date is defined and the end
start = ee.Date('2007-01-01')
dateRange = ee.DateRange(start, start.advance(13, 'year'))

'Satellite data:'
# Import LST image collection.
modis = ee.ImageCollection('MODIS/006/MOD11A1')
# Filter the LST collection to include only images intersecting the desired date range.
mod11a2 = modis.filterDate(dateRange)
# Select only the 1km day LST data band.
modLSTday = mod11a2.select('LST_Day_1km')
# Scale to Kelvin and convert to Celsius, set image acquisition time.

def func_rjp(img):
  return img \
    .multiply(0.02) \
    .subtract(273.15) \
    .copyProperties(img, ['system:time_start'])

modLSTc = modLSTday.map(func_rjp)

### Extract temperature for each municipality:

In [12]:
def func_hfv (image):
  date = image.date().format('yyyy-MM-dd')
  value = image.reduceRegion(
    reducer= ee.Reducer.mean(), 
    geometry= Municipality.geometry(), 
    scale= 1000).get('LST_Day_1km')
  return ee.Feature(None, {'value': value, 'date': date})

# Get time series data:
def get_time_series(Municipality):
  timeSeries = modLSTc.map(func_hfv)
  return timeSeries

In [13]:
total = Codes.size().getInfo()

for i in range(0, total, 1):
  code = Codes.getString(i).getInfo()
  print(code)
  # Apply filter where municipalty code equals code.
  Municipality = dataset.filter(ee.Filter.eq('MPIOS', code))
  timeSeries = get_time_series(Municipality)

  task = ee.batch.Export.table.toDrive(** {
    'collection': timeSeries,
    'folder': 'GEE_temperature',
    'description': 'temperature_' + code,
    'fileFormat': 'CSV',
    'selectors': ['value', 'date']
  })
  
  task.start()

print("Done!")

08001
08573
08832
08296
08758
08433
08372
08078
08634
08558
08549
08849
08685
08520
08638
08560
08421
08606
08141
08436
08137
08675
08770
13673
13001
13222
13683
13873
13647
13760
13620
13836
13052
13433
13838
13248
13657
13442
13654
13244
13430
13780
13468
13188
13650
13440
13549
13300
13268
13667
13074
13030
13006
13810
13458
13655
13062
13140
13894
13212
20001
20570
20621
20443
20238
20750
20013
20060
20045
20250
20400
20178
20032
20175
20228
20787
20517
20550
44420
44035
44847
44560
44430
44001
44090
44378
44078
44098
44279
44650
44874
44110
44855
47707
47545
47692
47703
47318
47245
47980
47030
47205
47960
47660
47460
47720
47001
47189
47745
47570
47053
47268
47605
47551
47288
47675
47258
47161
47541
47170
47798
47555
47058
70233
70820
70221
70508
70230
70823
70204
70717
70418
70473
70110
70001
70742
70523
70702
70215
70670
70235
70678
70124
70400
70708
70713
70771
70429
70265
88001
88000
88564
11001
15572
15507
15531
15632
15681
15832
15176
15106
15776
15442
15580
15808
15131
1567